In [23]:
df_1min

,timestamp,open,high,low,close,volume
0,2020-02-05 09:15:00+05:30,1234.90,1234.90,1230.00,1231.85,177538
1,2020-02-05 09:16:00+05:30,1231.95,1231.95,1230.00,1230.65,64165
2,2020-02-05 09:17:00+05:30,1230.55,1233.50,1230.00,1233.40,47079
3,2020-02-05 09:18:00+05:30,1233.40,1234.20,1232.70,1233.25,36476
4,2020-02-05 09:19:00+05:30,1232.95,1234.50,1232.80,1233.60,32442
...,...,...,...,...,...,...
253502,2022-11-01 11:06:00+05:30,1527.00,1527.25,1526.00,1526.65,13314
253503,2022-11-01 11:07:00+05:30,1526.60,1527.10,1525.40,1526.05,15394
253504,2022-11-01 11:08:00+05:30,1526.05,1527.00,1525.85,1526.70,8906
253505,2022-11-01 11:09:00+05:30,1526.75,1527.00,1526.70,1526.95,12439


In [17]:
df_5min

,open,high,low,close,volume
timestamp,,,,,
2020-02-05 09:15:00+05:30,1234.90,1234.90,1230.00,1233.60,357700
2020-02-05 09:20:00+05:30,1233.60,1236.95,1233.60,1235.60,141071
2020-02-05 09:25:00+05:30,1235.65,1236.70,1235.20,1236.65,148869
2020-02-05 09:30:00+05:30,1236.25,1238.90,1235.90,1237.70,96068
2020-02-05 09:35:00+05:30,1237.80,1239.25,1236.50,1237.25,81881
...,...,...,...,...,...
2022-11-01 10:50:00+05:30,1527.00,1528.05,1525.25,1527.05,101621
2022-11-01 10:55:00+05:30,1527.60,1529.00,1526.40,1526.85,157410
2022-11-01 11:00:00+05:30,1526.80,1527.90,1525.20,1526.80,82026


In [18]:
df_15min

,open,high,low,close,volume
timestamp,,,,,
2020-02-05 09:15:00+05:30,1234.90,1236.95,1230.00,1236.65,647640
2020-02-05 09:30:00+05:30,1236.25,1239.25,1235.90,1237.95,290928
2020-02-05 09:45:00+05:30,1237.95,1238.80,1235.60,1238.50,546720
2020-02-05 10:00:00+05:30,1238.50,1240.00,1237.30,1239.05,418092
2020-02-05 10:15:00+05:30,1239.05,1240.25,1237.30,1239.55,296474
...,...,...,...,...,...
2022-11-01 10:00:00+05:30,1521.55,1525.85,1520.15,1525.40,580172
2022-11-01 10:15:00+05:30,1525.45,1525.75,1522.55,1522.60,267462
2022-11-01 10:30:00+05:30,1523.15,1528.45,1520.25,1528.00,602687


In [2]:
import pandas as pd
import talib
data_1min = pd.read_csv('HDFCBANK.csv')

# Step 1: Resample df_1min to 5-minute timeframe
data_1min['timestamp'] = pd.to_datetime(data_1min['date'])
data_1min = data_1min[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
data_1min.set_index('timestamp', inplace=True)
data_1min.reset_index('timestamp', inplace=True)
# Resample to 5-minute and 15-minute timeframes
data_5min = data_1min.resample('5T', on='timestamp').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()

data_15min = data_1min.resample('15T', on='timestamp').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum'
}).dropna()

# Merge the dataframes on timestamp, filling missing values with the last available value
merged_data = pd.merge(data_1min, data_5min, on='timestamp', how='left', suffixes=('_1min', '_5min')).fillna(method='bfill')
data_15min.columns = [col + '_15min' for col in data_15min.columns]
merged_data = pd.merge(merged_data, data_15min, on='timestamp', how='left', suffixes=('', '_15min')).fillna(method='bfill')

merged_data['ema1min'] = talib.EMA(merged_data['close_1min'], timeperiod=12)
merged_data['ema5min'] = talib.EMA(merged_data['close_5min'], timeperiod=12)
merged_data['ema15min'] = talib.EMA(merged_data['close_15min'], timeperiod=12)
merged_data.dropna(inplace=True)
merged_data.head(50)

# data_1min

,timestamp,open_1min,high_1min,low_1min,close_1min,volume_1min,open_5min,high_5min,low_5min,close_5min,volume_5min,open_15min,high_15min,low_15min,close_15min,volume_15min,ema1min,ema5min,ema15min
11,2020-02-05 09:26:00+05:30,1236.10,1236.50,1236.00,1236.30,19067,1236.25,1238.90,1235.90,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1234.454167,1236.045833,1237.841667
12,2020-02-05 09:27:00+05:30,1236.25,1236.50,1235.50,1235.90,43421,1236.25,1238.90,1235.90,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1234.676603,1236.300321,1237.858333
13,2020-02-05 09:28:00+05:30,1235.90,1236.15,1235.20,1236.00,46040,1236.25,1238.90,1235.90,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1234.880202,1236.515656,1237.872436
14,2020-02-05 09:29:00+05:30,1236.00,1236.70,1235.95,1236.65,13857,1236.25,1238.90,1235.90,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1235.152479,1236.697863,1237.884369
15,2020-02-05 09:30:00+05:30,1236.25,1236.60,1235.90,1236.45,17771,1236.25,1238.90,1235.90,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1235.352097,1236.852038,1237.894466
16,2020-02-05 09:31:00+05:30,1236.50,1237.60,1236.00,1237.10,21341,1237.80,1239.25,1236.50,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1235.621006,1236.913263,1237.987625
17,2020-02-05 09:32:00+05:30,1236.95,1238.75,1236.95,1238.60,17237,1237.80,1239.25,1236.50,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1236.079312,1236.965068,1238.066452
18,2020-02-05 09:33:00+05:30,1238.50,1238.90,1238.00,1238.20,20728,1237.80,1239.25,1236.50,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1236.405572,1237.008904,1238.133152
19,2020-02-05 09:34:00+05:30,1238.25,1238.40,1237.70,1237.70,18991,1237.80,1239.25,1236.50,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1236.604715,1237.045996,1238.189590
20,2020-02-05 09:35:00+05:30,1237.80,1239.25,1237.70,1238.85,13862,1237.80,1239.25,1236.50,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1236.950143,1237.077381,1238.237345


In [3]:
merged_data.columns

Index(['timestamp', 'open_1min', 'high_1min', 'low_1min', 'close_1min',
       'volume_1min', 'open_5min', 'high_5min', 'low_5min', 'close_5min',
       'volume_5min', 'open_15min', 'high_15min', 'low_15min', 'close_15min',
       'volume_15min', 'ema1min', 'ema5min', 'ema15min'],
      dtype='object')

In [13]:
import pandas as pd
import talib

def resample_data(df, timeframe):
    df['timestamp'] = pd.to_datetime(df['date'])
    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    df.set_index('timestamp', inplace=True)
    df.reset_index('timestamp', inplace=True)
    
    resampled_data = df.resample(timeframe, on='timestamp').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()
    
    return resampled_data

def merge_data(data_1min, data_resampled, timeframe, suffixOne, suffixTwo):
    merged_data = pd.merge(data_1min, data_resampled, on='timestamp', how='left', suffixes=(suffixOne, suffixTwo)).fillna(method='bfill')
    return merged_data

def calculate_ema(df, column_name, timeframe):
    df[f'ema{timeframe}'] = talib.EMA(df[column_name], timeperiod=12)
    return df

data_1min = pd.read_csv('HDFCBANK.csv')

data_5min = resample_data(data_1min, '5T')
data_15min = resample_data(data_1min, '15T')

merged_data = merge_data(data_1min, data_5min, '5T', '_1min', '_5min')
data_15min.columns = [col + '_15min' for col in data_15min.columns]
merged_data = merge_data(merged_data, data_15min, '15T', '', '_15min')

merged_data = calculate_ema(merged_data, 'close_1min', '1')
merged_data = calculate_ema(merged_data, 'close_5min', '5')
merged_data = calculate_ema(merged_data, 'close_15min', '15')

merged_data.dropna(inplace=True)
merged_data.head(50)


,Unnamed: 0,date,open_1min,high_1min,low_1min,close_1min,volume_1min,timestamp,open_5min,high_5min,...,close_5min,volume_5min,open_15min,high_15min,low_15min,close_15min,volume_15min,ema1,ema5,ema15
11,11,2020-02-05 09:26:00+05:30,1236.10,1236.50,1236.00,1236.30,19067,2020-02-05 09:26:00+05:30,1236.25,1238.90,...,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1234.454167,1236.045833,1237.841667
12,12,2020-02-05 09:27:00+05:30,1236.25,1236.50,1235.50,1235.90,43421,2020-02-05 09:27:00+05:30,1236.25,1238.90,...,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1234.676603,1236.300321,1237.858333
13,13,2020-02-05 09:28:00+05:30,1235.90,1236.15,1235.20,1236.00,46040,2020-02-05 09:28:00+05:30,1236.25,1238.90,...,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1234.880202,1236.515656,1237.872436
14,14,2020-02-05 09:29:00+05:30,1236.00,1236.70,1235.95,1236.65,13857,2020-02-05 09:29:00+05:30,1236.25,1238.90,...,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1235.152479,1236.697863,1237.884369
15,15,2020-02-05 09:30:00+05:30,1236.25,1236.60,1235.90,1236.45,17771,2020-02-05 09:30:00+05:30,1236.25,1238.90,...,1237.70,96068.0,1236.25,1239.25,1235.9,1237.95,290928.0,1235.352097,1236.852038,1237.894466
16,16,2020-02-05 09:31:00+05:30,1236.50,1237.60,1236.00,1237.10,21341,2020-02-05 09:31:00+05:30,1237.80,1239.25,...,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1235.621006,1236.913263,1237.987625
17,17,2020-02-05 09:32:00+05:30,1236.95,1238.75,1236.95,1238.60,17237,2020-02-05 09:32:00+05:30,1237.80,1239.25,...,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1236.079312,1236.965068,1238.066452
18,18,2020-02-05 09:33:00+05:30,1238.50,1238.90,1238.00,1238.20,20728,2020-02-05 09:33:00+05:30,1237.80,1239.25,...,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1236.405572,1237.008904,1238.133152
19,19,2020-02-05 09:34:00+05:30,1238.25,1238.40,1237.70,1237.70,18991,2020-02-05 09:34:00+05:30,1237.80,1239.25,...,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1236.604715,1237.045996,1238.189590
20,20,2020-02-05 09:35:00+05:30,1237.80,1239.25,1237.70,1238.85,13862,2020-02-05 09:35:00+05:30,1237.80,1239.25,...,1237.25,81881.0,1237.95,1238.80,1235.6,1238.50,546720.0,1236.950143,1237.077381,1238.237345


In [11]:
merged_data.columns

Index(['Unnamed: 0', 'date', 'open_1min', 'high_1min', 'low_1min',
       'close_1min', 'volume_1min', 'timestamp', 'open_5min', 'high_5min',
       'low_5min', 'close_5min', 'volume_5min', 'open_15min', 'high_15min',
       'low_15min', 'close_15min', 'volume_15min', 'emaclose_1min',
       'emaclose_5min', 'emaclose_15min'],
      dtype='object')

In [ ]:
for index, this_candle in merged_data.iterrows():
    